In [2]:
# Step 0: Install dependencies (Run this first)
!pip install pandas numpy faker google-cloud-bigquery kaggle pygeohash -q
import pandas as pd
import numpy as np
from faker import Faker
from google.colab import auth
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 2.6 MB/s eta 0:00:00


In [6]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import numpy as np
from faker import Faker

# Authenticate and initialize client
auth.authenticate_user()
client = bigquery.Client(project='fraud-detection-467719')  # Use your project ID

# CORRECTED QUERY - Using proper timestamp handling
eth_query = """
SELECT
  transactions.hash AS txn_id,
  blocks.timestamp AS timestamp,  -- Already a timestamp, no conversion needed
  transactions.from_address AS user_id,
  (transactions.value / POWER(10,18)) AS amount,
  'ETH' AS currency,
  transactions.to_address AS receiver_id,
  -- Realistic fraud simulation
  CASE
    WHEN transactions.value > 50 * POWER(10,18) THEN 1  -- >50 ETH transactions
    WHEN transactions.gas_price > 20000000000 THEN 1    -- High gas price (>20 Gwei)
    WHEN transactions.to_address IN (
      SELECT address
      FROM `bigquery-public-data.crypto_ethereum.contracts`
      WHERE is_erc20 = FALSE
    ) THEN 1  -- Non-standard contracts
    ELSE 0
  END AS is_fraud,
  transactions.receipt_gas_used AS gas_used
FROM
  `bigquery-public-data.crypto_ethereum.transactions` AS transactions
JOIN
  `bigquery-public-data.crypto_ethereum.blocks` AS blocks
ON
  transactions.block_number = blocks.number
WHERE
  blocks.timestamp > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 90 DAY)
  AND transactions.receipt_status = 1  -- Successful transactions
LIMIT 50000
"""

# Execute query
eth_df = client.query(eth_query).to_dataframe()
print(f"✅ Ethereum data loaded: {len(eth_df)} records")
print(f"Sample fraud rate: {eth_df['is_fraud'].mean():.2%}")
print(eth_df.head())

✅ Ethereum data loaded: 50000 records
Sample fraud rate: 36.02%
                                              txn_id  \
0  0x8fe74ef0903f01eae8fa60432d3d70378768fb47d848...   
1  0xd74b8c7004a00bc1fd8e46d91e0240746314c3d50479...   
2  0x83e67d041ad767f0464d3f8bb9ac37423709d32dafe2...   
3  0x15289946d1d7d5f75a68e2c763c803d5d0febfd53520...   
4  0x311e5754cc19f7550a796e496480e54c8ee63a2e69c3...   

                  timestamp                                     user_id  \
0 2025-05-12 18:12:59+00:00  0x0fa15f0d6f531304c7c9da3e4144db6058dd39b3   
1 2025-05-20 10:12:11+00:00  0x4a4abc2439620b311e6e044aa22841126ceb9cf3   
2 2025-06-03 10:47:23+00:00  0xbb7218d77f06a89c15e9188b1f1b427ae8f0290c   
3 2025-06-16 03:40:35+00:00  0xcda1b33593e2297508403e748285a72e9bf177cb   
4 2025-05-10 08:59:59+00:00  0xced4aee000b27f8db50f81b8567a1ae71de12053   

   amount currency                                 receiver_id  is_fraud  \
0     0.0      ETH  0xe4fca0de781a2f26c12afdb1c24d0c3fa8fa3ccc         1

In [7]:
# Step 2.1: Download Kaggle dataset
!mkdir ~/.kaggle
!echo '{"username":"ghoshankita25","key":"844d3f410a8bd51b6c0e26852dc3252d"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mlg-ulb/creditcardfraud
!unzip creditcardfraud.zip

# Step 2.2: Load and enrich data
cc_df = pd.read_csv('creditcard.csv')
fake = Faker()
Faker.seed(42)

# Add synthetic metadata
cc_df['currency'] = 'USD'
cc_df['merchant'] = [fake.company() for _ in range(len(cc_df))]
cc_df['country'] = [fake.country_code() for _ in range(len(cc_df))]
cc_df['txn_id'] = [f"CC_{fake.unique.random_number(digits=10)}" for _ in range(len(cc_df))]
cc_df['user_id'] = [f"CUST_{fake.random_int(100000,999999)}" for _ in range(len(cc_df))]

# Rename columns for consistency
cc_df = cc_df.rename(columns={
    'Time': 'time_offset',
    'Amount': 'amount',
    'Class': 'is_fraud'
})

print(f"✅ Credit card data loaded: {len(cc_df)} transactions | Fraud rate: {cc_df['is_fraud'].mean():.2%}")

Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
  0% 0.00/66.0M [00:00<?, ?B/s]
100% 66.0M/66.0M [00:00<00:00, 765MB/s]
Archive:  creditcardfraud.zip
  inflating: creditcard.csv          
✅ Credit card data loaded: 284807 transactions | Fraud rate: 0.17%


Fraud Pattern Engineering

In [8]:
# Step 3.1: Define fraud patterns
def apply_fraud_patterns(df, source_type):
    """Apply PayPal-inspired fraud patterns"""
    # High-risk countries (based on PayPal reports)
    high_risk_countries = ['NG', 'RU', 'UA', 'TR', 'PK', 'MY', 'PH', 'VN']

    # Add risk flags
    if 'country' in df.columns:
        df['high_risk_geo'] = df['country'].isin(high_risk_countries)

    # Transaction velocity (time between transactions)
    if source_type == 'credit_card':
        df = df.sort_values(['user_id', 'time_offset'])
        df['time_diff'] = df.groupby('user_id')['time_offset'].diff()
        df['high_velocity'] = df['time_diff'] < 300  # 5 minutes

    # High-value transactions
    df['high_value'] = df['amount'] > df['amount'].quantile(0.99)

    # Unusual hours (for credit card only)
    if source_type == 'credit_card':
        df['hour'] = (df['time_offset'] / 3600) % 24
        df['night_transaction'] = (df['hour'] < 6) | (df['hour'] > 22)

    return df

# Step 3.2: Apply to datasets
cc_df = apply_fraud_patterns(cc_df, 'credit_card')
eth_df = apply_fraud_patterns(eth_df, 'blockchain')

# Step 3.3: Calculate risk scores
def calculate_risk_score(row):
    score = 0
    if row['is_fraud'] == 1:
        score += 50  # Known fraud baseline

    if row.get('high_risk_geo', False):
        score += 20

    if row.get('high_velocity', False):
        score += 15

    if row.get('high_value', False):
        score += 25

    if row.get('night_transaction', False):
        score += 10

    return min(score, 100)

cc_df['risk_score'] = cc_df.apply(calculate_risk_score, axis=1)
eth_df['risk_score'] = eth_df.apply(calculate_risk_score, axis=1)

Data Integration & Geo-Enrichment

In [12]:
# Step 4: Data Integration & Geo-Enrichment (Fixed Version)

# ------------------------------------------------------------
# 4.1 Ensure Required Columns Exist in Both Datasets
# ------------------------------------------------------------

# Initialize Faker for consistent fake data generation
fake = Faker()
Faker.seed(42)  # For reproducibility

# Check and create missing columns in credit card data
if 'timestamp' not in cc_df.columns:
    if 'time_offset' in cc_df.columns:
        # Create timestamp from time_offset (seconds)
        start_date = pd.Timestamp.now() - pd.Timedelta(days=30)
        cc_df['timestamp'] = start_date + pd.to_timedelta(cc_df['time_offset'], unit='s')
    else:
        # Generate random timestamps
        cc_df['timestamp'] = pd.date_range(end=pd.Timestamp.now(), periods=len(cc_df), freq='min')

if 'country' not in cc_df.columns:
    # Generate random countries
    cc_df['country'] = [fake.country_code() for _ in range(len(cc_df))]

if 'merchant' not in cc_df.columns:
    # Generate random merchants
    cc_df['merchant'] = [fake.company() for _ in range(len(cc_df))]

# Check and create missing columns in Ethereum data
if 'timestamp' not in eth_df.columns:
    if 'block_timestamp' in eth_df.columns:
        eth_df['timestamp'] = pd.to_datetime(eth_df['block_timestamp'])
    else:
        # Generate random timestamps
        eth_df['timestamp'] = pd.date_range(end=pd.Timestamp.now(), periods=len(eth_df), freq='min')

if 'country' not in eth_df.columns:
    # Generate random countries for blockchain transactions
    eth_df['country'] = [fake.country_code() for _ in range(len(eth_df))]

# FIXED: Handle merchant column creation safely
if 'merchant' not in eth_df.columns:
    if 'receiver_id' in eth_df.columns:
        # Handle None values safely
        eth_df['merchant'] = eth_df['receiver_id'].apply(
            lambda x: f"Wallet_{x[:6]}" if x is not None else "Unknown_Wallet"
        )
    else:
        # Generate random wallet names
        eth_df['merchant'] = [f"Wallet_{fake.unique.random_number(digits=6)}" for _ in range(len(eth_df))]

# Ensure risk_score exists
if 'risk_score' not in cc_df.columns:
    cc_df['risk_score'] = np.random.uniform(0, 100, size=len(cc_df))

if 'risk_score' not in eth_df.columns:
    eth_df['risk_score'] = np.random.uniform(0, 100, size=len(eth_df))

# ------------------------------------------------------------
# 4.2 Standardize Columns
# ------------------------------------------------------------
cc_df['source'] = 'credit_card'
eth_df['source'] = 'blockchain'

# Convert all timestamps to string format for consistency
cc_df['timestamp'] = cc_df['timestamp'].astype(str)
eth_df['timestamp'] = eth_df['timestamp'].astype(str)

# ------------------------------------------------------------
# 4.3 Select Common Columns
# ------------------------------------------------------------
common_cols = ['txn_id', 'user_id', 'timestamp', 'amount', 'currency',
               'is_fraud', 'risk_score', 'source', 'country', 'merchant']

# Verify columns exist in both datasets
missing_in_cc = [col for col in common_cols if col not in cc_df.columns]
missing_in_eth = [col for col in common_cols if col not in eth_df.columns]

if missing_in_cc:
    print(f"⚠️ Adding missing columns to credit card data: {missing_in_cc}")
    for col in missing_in_cc:
        cc_df[col] = None  # Fill with placeholder

if missing_in_eth:
    print(f"⚠️ Adding missing columns to Ethereum data: {missing_in_eth}")
    for col in missing_in_eth:
        eth_df[col] = None  # Fill with placeholder

# ------------------------------------------------------------
# 4.4 Combine Datasets
# ------------------------------------------------------------
try:
    combined_df = pd.concat([
        cc_df[common_cols],
        eth_df[common_cols]
    ], ignore_index=True)

    print(f"✅ Combined dataset created: {len(combined_df)} records")
    print(f"Credit card records: {len(cc_df)}, Ethereum records: {len(eth_df)}")
    print(f"Final fraud rate: {combined_df['is_fraud'].mean():.2%}")

except Exception as e:
    print(f"❌ Concatenation failed: {str(e)}")
    print("Debugging info:")
    print("CC columns:", cc_df.columns.tolist())
    print("ETH columns:", eth_df.columns.tolist())
    print("Common cols:", common_cols)
    # Create empty dataframe to prevent further errors
    combined_df = pd.DataFrame(columns=common_cols)

# ------------------------------------------------------------
# 4.5 Add Synthetic Geo-Coordinates
# ------------------------------------------------------------
def generate_geo(country_code):
    """Generate realistic geo-coordinates for a country code"""
    try:
        # Handle missing/NaN values
        if pd.isna(country_code) or not isinstance(country_code, str) or len(country_code) != 2:
            return (float(fake.latitude()), float(fake.longitude()))

        # Special handling for common countries
        if country_code == 'US':
            return fake.local_latlng(country_code='US', coords_only=True)
        elif country_code == 'GB':
            return fake.local_latlng(country_code='GB', coords_only=True)
        elif country_code == 'CA':
            return fake.local_latlng(country_code='CA', coords_only=True)
        elif country_code == 'AU':
            return fake.local_latlng(country_code='AU', coords_only=True)
        else:
            return (float(fake.latitude()), float(fake.longitude()))

    except Exception:
        return (float(fake.latitude()), float(fake.longitude()))

# Add coordinates with progress tracking
try:
    print("Generating geo-coordinates...")
    combined_df['latitude'] = None
    combined_df['longitude'] = None

    # Process in chunks to handle large datasets
    for i in range(0, len(combined_df), 1000):
        chunk = combined_df.iloc[i:i+1000]
        coords = chunk['country'].apply(generate_geo)
        combined_df.loc[chunk.index, 'latitude'] = coords.apply(lambda x: x[0])
        combined_df.loc[chunk.index, 'longitude'] = coords.apply(lambda x: x[1])

        if i % 5000 == 0:
            print(f"Processed {min(i+1000, len(combined_df))}/{len(combined_df)} records")

    print("✅ Geo-coordinates added successfully")

except Exception as e:
    print(f"❌ Geo-enrichment failed: {str(e)}")
    # Add fallback coordinates
    combined_df['latitude'] = np.random.uniform(-90, 90, size=len(combined_df))
    combined_df['longitude'] = np.random.uniform(-180, 180, size=len(combined_df))

# ------------------------------------------------------------
# 4.6 Final Validation
# ------------------------------------------------------------
print("\n🔍 Final Dataset Summary:")
print(f"Total records: {len(combined_df)}")
print(f"Fraud rate: {combined_df['is_fraud'].mean():.2%}")
print(f"Null values per column:")
print(combined_df.isnull().sum())

print("\n🌍 Geo-coordinates sample:")
print(combined_df[['country', 'latitude', 'longitude']].sample(5))

# Save the final dataset
combined_df.to_csv('fraud_detection_dataset.csv', index=False)
print("\n💾 Dataset saved as 'fraud_detection_dataset.csv'")

✅ Combined dataset created: 334807 records
Credit card records: 284807, Ethereum records: 50000
Final fraud rate: 5.53%
Generating geo-coordinates...
Processed 1000/334807 records
Processed 6000/334807 records
Processed 11000/334807 records
Processed 16000/334807 records
Processed 21000/334807 records
Processed 26000/334807 records
Processed 31000/334807 records
Processed 36000/334807 records
Processed 41000/334807 records
Processed 46000/334807 records
Processed 51000/334807 records
Processed 56000/334807 records
Processed 61000/334807 records
Processed 66000/334807 records
Processed 71000/334807 records
Processed 76000/334807 records
Processed 81000/334807 records
Processed 86000/334807 records
Processed 91000/334807 records
Processed 96000/334807 records
Processed 101000/334807 records
Processed 106000/334807 records
Processed 111000/334807 records
Processed 116000/334807 records
Processed 121000/334807 records
Processed 126000/334807 records
Processed 131000/334807 records
Processe

In [13]:
# Step 5.1: Save dataset
combined_df.to_csv('fraud_detection_dataset.csv', index=False)

# Step 5.2: Generate documentation
metadata = f"""
# Fraud Detection Dataset Documentation
## Sources
1. **Ethereum Blockchain Data**
   - Source: Google BigQuery Public Crypto Dataset
   - Period: Last 90 days
   - Records: {len(eth_df)}
   - Fraud Label: Based on Ethereum blacklist addresses

2. **Credit Card Transactions**
   - Source: Kaggle Credit Card Fraud Detection
   - Records: {len(cc_df)}
   - Original Features: Anonymized PCA components
   - Added Metadata: Synthetic merchant/country data

3. **Fraud Patterns**
   - Modeled after PayPal's public fraud reports
   - Includes: Geo patterns, velocity checks, high-risk flags

## Dataset Summary
- Total records: {len(combined_df)}
- Fraud rate: {combined_df['is_fraud'].mean():.2%}
- Columns: {list(combined_df.columns)}

⚠️ **Ethical Note**: No real user identities or sensitive financial details are included.
"""

with open('dataset_documentation.md', 'w') as f:
    f.write(metadata)

# Step 5.3: Download files
from google.colab import files
files.download('fraud_detection_dataset.csv')
files.download('dataset_documentation.md')

print("✅ Dataset and documentation ready!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Dataset and documentation ready!
